Solving Package delivery using single-agent PPO with a naive feature representation learning: concatenante all the feature in to a single state vector, and multiple robot actions as a multi discrete distribution.

In [13]:
%%capture
!git clone https://github.com/cuongtv312/marl-delivery.git
%cd marl-delivery
!pip install -r requirements.txt

In [2]:
%%capture
!pip install stable-baselines3

In [3]:
from env import Environment
import gymnasium as gym
from gymnasium import spaces
import numpy as np

In [6]:
# TODO: Modify this one to add more information to the Agents
def convert_state(state):
    ret_state = {}
    # Add time step information
    ret_state["time_step"] = np.array([state["time_step"]]).astype(np.float32).flatten()

    # Add map information
    ret_state["map"] = np.array(state["map"]).astype(np.float32).flatten()

    # Add robot information
    ret_state["robots"] = np.array(state["robots"]).astype(np.float32).flatten()

    # Add package information (limit to 100 packages)
    ret_state["packages"] = np.array(state["packages"]).astype(np.float32).flatten()[:100]
    if len(ret_state["packages"]) < 100:
        ret_state["packages"] = np.concatenate((ret_state["packages"], np.zeros(100 - len(ret_state["packages"]))))

    # Combine all information into a single state vector
    return np.concatenate(list(ret_state.values()))

In [7]:
# TODO: Modify this one to make the agent learn faster

def reward_shaping(r, env, state, action):
    # Base reward from the environment
    shaped_reward = r

    # Add a penalty for idle actions (e.g., staying in the same position)
    if action[0] == 'S':
        shaped_reward -= 0.1

    # Add a bonus for delivering packages on time
    for package in state['packages']:
        if package[5] <= state['time_step'] <= package[6]:  # Within delivery window
            shaped_reward += 1.0

    # Add a penalty for moving further away from the target
    for robot, act in zip(state['robots'], action):
        if act[0] in ['L', 'R', 'U', 'D']:
            target = next((p for p in state['packages'] if p[0] == robot[2]), None)
            if target:
                current_dist = abs(robot[0] - target[3]) + abs(robot[1] - target[4])
                new_pos = env.compute_new_position((robot[0], robot[1]), act[0])
                new_dist = abs(new_pos[0] - target[3]) + abs(new_pos[1] - target[4])
                if new_dist > current_dist:
                    shaped_reward -= 0.5  # Penalize moving away from the target

    return shaped_reward

In [8]:
# Avoid to modify the Env class,
# If it is neccessary, you should describe those changes clearly in report and code
class Env(gym.Env):
    def __init__(self, *args, **kwargs):
        super(Env, self).__init__()
        self.env = Environment(*args, **kwargs)

        self.action_space = spaces.multi_discrete.MultiDiscrete([5, 3]*self.env.n_robots)


        self.prev_state = self.env.reset()
        first_state=convert_state(self.prev_state)
        # Define observation space as a dictionary

        self.observation_space = spaces.Box(low=0, high=100, shape=first_state.shape, dtype=np.float32)


        from sklearn.preprocessing import LabelEncoder
        self.le1, self.le2= LabelEncoder(), LabelEncoder()
        self.le1.fit(['S', 'L', 'R', 'U', 'D'])
        self.le2.fit(['0','1', '2'])

    def reset(self, *args, **kwargs):
        self.prev_state = self.env.reset()
        return convert_state(self.prev_state), {}

    def render(self, *args, **kwargs):
        return self.env.render()

    def step(self, action):
        ret = []
        ret.append(self.le1.inverse_transform(action.reshape(-1, 2).T[0]))
        ret.append(self.le2.inverse_transform(action.reshape(-1, 2).T[1]))
        action = list(zip(*ret))

        # You should not modify the infos object
        s, r, done, infos = self.env.step(action)
        new_r = reward_shaping(r, self.env, self.prev_state, action)
        self.prev_state = s
        return convert_state(s), new_r, \
            done, False, infos

In [26]:
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import EvalCallback


# Parallel environments
vec_env = make_vec_env(lambda: Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10), n_envs=10)
eval_env = Monitor(Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10), "ppo_delivery")

eval_callback = EvalCallback(eval_env, best_model_save_path="./best_model/",
                             log_path="./logs/", eval_freq=5000,
                             deterministic=True, render=False)

model = PPO("MlpPolicy", vec_env, verbose=1)
model.learn(total_timesteps=10000, callback=eval_callback)
model.save("ppo_delivery")


Using cpu device
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 100      |
|    ep_rew_mean     | -2.68    |
| time/              |          |
|    fps             | 1017     |
|    iterations      | 1        |
|    time_elapsed    | 20       |
|    total_timesteps | 20480    |
---------------------------------


In [27]:
obs,_ = eval_env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, _, info = eval_env.step(action)
    #print('='*10)
    #eval_env.unwrapped.env.render()
    if dones:
        break

print(info)

{'total_reward': -3.339999999999998, 'total_time_steps': 100, 'episode': {'r': -3.34, 'l': 100, 't': 101.408837}}


In [7]:
!pip freeze | grep stable_baselines3

stable_baselines3==2.6.0


# Implementing DQN for Package Delivery
This section implements the DQN algorithm for solving the package delivery problem, similar to the PPO implementation.

In [16]:
from dqn_agent import DQNAgent
import numpy as np

# Initialize environment
env = Env('map2.txt', 100, 5, 20, -0.01, 10., 1., 10)

# Initialize DQN agent
state_size = env.observation_space.shape[0]
action_size = int(np.prod(env.action_space.nvec))  # Flatten MultiDiscrete action space
agent = DQNAgent(state_size, action_size)

# Training parameters
n_episodes = 1000
batch_size = 32

# Training loop
for episode in range(n_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0

    while not done:
        actions = []
        robots = env.env.get_state()['robots']  # Access robots from the environment state
        for robot_idx in range(len(robots)):
            # Select action for each robot
            action = agent.get_action(state, robot_idx, training=True)
            actions.append(action)

        # Convert actions to NumPy array and encode using LabelEncoder
        actions = np.array(actions)
        actions[:, 0] = env.le1.transform(actions[:, 0])  # Encode first part of action
        actions[:, 1] = env.le2.transform(actions[:, 1])  # Encode second part of action

        # Take actions in environment
        next_state, reward, done, _, _ = env.step(actions)

        # Store experience and train
        agent.remember(state, actions, reward, next_state, done)
        agent.replay(batch_size)

        # Update state and reward
        state = next_state
        total_reward += reward

    # Update target network periodically
    if (episode + 1) % 10 == 0:
        agent.update_target_model()

    print(f"Episode {episode + 1}/{n_episodes}, Total Reward: {total_reward}")

# Save the trained model
agent.save('dqn_model.pth')

ValueError: y contains previously unseen labels: ['0' '1' '2' '4']